In [1]:
%load_ext autoreload
%autoreload 2

In [33]:
import numpy as np
from train import *
from tensorflow.keras import layers
from tensorflow.keras.layers import TimeDistributed, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
import kapre
from kapre.composed import get_melspectrogram_layer
import tensorflow as tf
import os

In [34]:
#Prepare for Training
train_data = np.load("stft_object_clipping.npy")
num_object = train_data.shape[0]
num_instance = train_data.shape[1]
dim_instance = train_data.shape[2:]
print("Number of Training Object:",num_object)
print("Number of Training instance for each object:",num_instance)
print("Dimension of each instance:",dim_instance)

Number of Training Object: 2
Number of Training instance for each object: 300
Dimension of each instance: (36, 234)


In [35]:
classes = sorted(os.listdir("wavfiles_ultramic"))
le = LabelEncoder()
le.fit(classes)

LabelEncoder()

In [36]:
#Create label for each instance
labels = []
for i in range(0,num_object):
    for j in range(0,num_instance):
        labels.append(classes[i])
labels = le.transform(labels)
train_data = train_data.reshape(num_object*num_instance,dim_instance[0],dim_instance[1])
print(train_data.shape)

(600, 36, 234)


In [37]:
trainset, valset, label_train, label_val = train_test_split(train_data,
                                                    labels,
                                                    test_size=0.2,
                                                    random_state=0)

In [38]:
class DataGenerator_npy(tf.keras.utils.Sequence):
    def __init__(self, data, labels, dim, n_classes,
                 batch_size=8, shuffle=True):
        self.data = data
        self.labels = labels
        self.dim = dim
        self.n_classes = n_classes
        self.batch_size = batch_size
        self.shuffle = True
        self.on_epoch_end()


    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))


    def __getitem__(self, index):
        # Where is the input of index?
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        data_tmp = [self.data[k] for k in indexes]
        labels = [self.labels[k] for k in indexes]

        # generate a batch of time data
        X = np.empty((self.batch_size, *self.dim, 1), dtype=np.float32)
        # The format of Y gonna be: Y[0,:] = 0,1/1,0
        Y = np.empty((self.batch_size, self.n_classes), dtype=np.float32)

        for i, (instance, label) in enumerate(zip(data_tmp, labels)):
            X[i,] = instance.reshape((*self.dim,1))
            Y[i,] = to_categorical(label, num_classes=self.n_classes)

        return X, Y


    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [39]:
tg = DataGenerator_npy(trainset, label_train, dim_instance,num_object)

In [40]:
vg = DataGenerator_npy(valset, label_val, dim_instance,num_object)

In [41]:
def Conv1D_ultramic(N_CLASSES=2, dim=(36,234)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = TimeDistributed(layers.Conv1D(8, kernel_size=(4), activation='tanh'), name='td_conv_1d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_1')(x)
    x = TimeDistributed(layers.Conv1D(16, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_2')(x)
    x = TimeDistributed(layers.Conv1D(32, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_3')(x)
    x = TimeDistributed(layers.Conv1D(64, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), name='max_pool_2d_4')(x)
    x = TimeDistributed(layers.Conv1D(128, kernel_size=(4), activation='relu'), name='td_conv_1d_relu_4')(x)
    x = layers.GlobalMaxPooling2D(name='global_max_pooling_2d')(x)
    x = layers.Dropout(rate=0.1, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='1d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def Conv2D_ultramic(N_CLASSES=2, dim=(36,234)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = layers.Conv2D(8, kernel_size=(7,7), activation='tanh', padding='same', name='conv2d_tanh')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_1')(x)
    x = layers.Conv2D(16, kernel_size=(5,5), activation='relu', padding='same', name='conv2d_relu_1')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_2')(x)
    x = layers.Conv2D(16, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_2')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_3')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_3')(x)
    x = layers.MaxPooling2D(pool_size=(2,2), padding='same', name='max_pool_2d_4')(x)
    x = layers.Conv2D(32, kernel_size=(3,3), activation='relu', padding='same', name='conv2d_relu_4')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(64, activation='relu', activity_regularizer=l2(0.001), name='dense')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='2d_convolution')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def LSTM_ultramic(N_CLASSES=2, dim=(36,234)):
    input = tf.keras.Input(shape=(*dim,1))
    x = LayerNormalization(axis=2, name='batch_norm')(input)
    x = TimeDistributed(layers.Reshape((-1,)), name='reshape')(x)
    s = TimeDistributed(layers.Dense(64, activation='tanh'),
                        name='td_dense_tanh')(x)
    x = layers.Bidirectional(layers.LSTM(32, return_sequences=True),
                             name='bidirectional_lstm')(s)
    x = layers.concatenate([s, x], axis=2, name='skip_connection')
    x = layers.Dense(64, activation='relu', name='dense_1_relu')(x)
    x = layers.MaxPooling1D(name='max_pool_1d')(x)
    x = layers.Dense(32, activation='relu', name='dense_2_relu')(x)
    x = layers.Flatten(name='flatten')(x)
    x = layers.Dropout(rate=0.2, name='dropout')(x)
    x = layers.Dense(32, activation='relu',
                         activity_regularizer=l2(0.001),
                         name='dense_3_relu')(x)
    o = layers.Dense(N_CLASSES, activation='softmax', name='softmax')(x)
    model = Model(inputs=input, outputs=o, name='long_short_term_memory')
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [48]:
model = LSTM_ultramic()
csv_path = os.path.join('logs', '{}_test.csv'.format("lstm"))

In [49]:
cp = ModelCheckpoint('models/{}.h5'.format("lstm_ultarmic"), monitor='val_loss',
                         save_best_only=True, save_weights_only=False,
                         mode='auto', save_freq='epoch', verbose=1)
csv_logger = CSVLogger(csv_path, append=False)
model.fit(tg, validation_data=vg,
              epochs=30, verbose=1,
              callbacks=[csv_logger, cp])

Epoch 1/30
55/60 [==========================>...] - ETA: 0s - loss: 0.3069 - accuracy: 0.8750
Epoch 00001: val_loss improved from inf to 0.06707, saving model to models\lstm_ultarmic.h5
60/60 [==============================] - 1s 13ms/step - loss: 0.2954 - accuracy: 0.8792 - val_loss: 0.0671 - val_accuracy: 0.9917
Epoch 2/30
55/60 [==========================>...] - ETA: 0s - loss: 0.1364 - accuracy: 0.9455
Epoch 00002: val_loss improved from 0.06707 to 0.05354, saving model to models\lstm_ultarmic.h5
60/60 [==============================] - 0s 7ms/step - loss: 0.1288 - accuracy: 0.9500 - val_loss: 0.0535 - val_accuracy: 1.0000
Epoch 3/30
55/60 [==========================>...] - ETA: 0s - loss: 0.0547 - accuracy: 0.9864
Epoch 00003: val_loss did not improve from 0.05354
60/60 [==============================] - 0s 6ms/step - loss: 0.0548 - accuracy: 0.9854 - val_loss: 0.0666 - val_accuracy: 0.9833
Epoch 4/30
54/60 [==========================>...] - ETA: 0s - loss: 0.0490 - accuracy: 0.98